Background: Dogecoin is a memecoin because it was created as a joke but has attracted a lot of attention when it ranked top 10 with ~33B USD market cap

Context: In Dogecoin's perspective, how does Elon Musk tweeting about Dogecoin influence the Google search engagement and crypto market value

Hypotheses

1. Elon Musk tweeting about Dogecoin heavily impacts the cryto price
    - Analyze the reaction of Dogecoin price 10 mins after Elon Musk tweeted about it
2. Elon Musk tweeting about Dogecoin heavily impacts the Google search engagement
    - Alanlyze possible delayed effect of Elon Musk's tweet on Dogecoin' Google search engagement

### Import Packages

In [73]:
import pandas as pd
import ccxt
from datetime import datetime,timedelta
import numpy as np
from pytrends.request import TrendReq
from dateutil import parser

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Read Dogecoin Tweets and Cyrpto Price to Merge

In [74]:
tweets_df = pd.read_csv('data/dogecoin_tweets.csv')
tweets_df['timestamp'] = pd.to_datetime(tweets_df['timestamp'])

dogecoin_price_df = pd.read_csv('data/dogecoin_price.csv')
dogecoin_price_df['timestamp'] = pd.to_datetime(dogecoin_price_df['timestamp'], unit='ms')

merge_df = pd.merge(left=dogecoin_price_df, right=tweets_df, how='left', on='timestamp')

In [83]:
# Proprocess the price data
# select the tweet_content col that is not null
twitter_index = merge_df[merge_df['tweet_content'].notnull()].index
merge_df.loc[twitter_index, 'tweet'] =True # If it is the time that Musk tweeted


# Calculate the change in price
merge_df['price_change'] = (merge_df['close']-merge_df['open'])/merge_df['open']



In [84]:
# Identify the timestamp with gaps
timestamp_gap_index = []
timestamp_col = merge_df["timestamp"]
for i, timestamp in enumerate(timestamp_col):
    if i!=0 and timestamp-timestamp_col[i-1]>timedelta(minutes=1):
        timestamp_gap_index.append(i)
merge_df.loc[timestamp_gap_index,'time_gap']=True
merge_df.loc[timestamp_gap_index,["price_change"]]=np.nan # Fill the gap with nan
merge_df.head()


,timestamp,open,high,low,close,volume,tweet_content,retweet_count,quote_count,like_count,tweet,price_change,time_gap
0,2020-12-20 08:30:00,0.003920,0.003930,0.003920,0.003922,1487104.0,NaN,NaN,NaN,NaN,NaN,0.000485,NaN
1,2020-12-20 08:31:00,0.003925,0.003925,0.003922,0.003922,181327.0,NaN,NaN,NaN,NaN,NaN,-0.000713,NaN
2,2020-12-20 08:32:00,0.003922,0.003922,0.003922,0.003922,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,2020-12-20 08:33:00,0.003922,0.003922,0.003922,0.003922,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
4,2020-12-20 08:34:00,0.003922,0.003923,0.003922,0.003922,167833.0,NaN,NaN,NaN,NaN,NaN,0.000051,NaN


In [89]:
# Select the index between two time_gap which before the tweet time
for count,index in enumerate(timestamp_gap_index):
    price_change_before_twitter = []
    price_change_after_twitter = []
    if count!=0:
        dfm_temp = merge_df.loc[timestamp_gap_index[count-1]:index-1]
    else:
        dfm_temp = merge_df.loc[0:index-1]
    # Analysis
    # Find the first tweet index 
    first_tweet_index = dfm_temp[dfm_temp['tweet']==True].index[0]
    # Find the last tweet index
    last_tweet_index = dfm_temp[dfm_temp['tweet']==True].index[-1]
    # Find the first price change before the first tweet
    before_price_change_index = dfm_temp.loc[dfm_temp.index[0]:first_tweet_index]['price_change']
    # Find the last price change after the last tweet
    after_price_change_index = dfm_temp.loc[last_tweet_index:]['price_change']

IndexError: index 0 is out of bounds for axis 0 with size 0

In [106]:
merge_df[0:140]

,timestamp,open,high,low,close,volume,tweet_content,retweet_count,quote_count,like_count,tweet,price_change,time_gap
0,2020-12-20 08:30:00,0.003920,0.003930,0.003920,0.003922,1487104.0,NaN,NaN,NaN,NaN,NaN,0.000485,NaN
1,2020-12-20 08:31:00,0.003925,0.003925,0.003922,0.003922,181327.0,NaN,NaN,NaN,NaN,NaN,-0.000713,NaN
2,2020-12-20 08:32:00,0.003922,0.003922,0.003922,0.003922,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,2020-12-20 08:33:00,0.003922,0.003922,0.003922,0.003922,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
4,2020-12-20 08:34:00,0.003922,0.003923,0.003922,0.003922,167833.0,NaN,NaN,NaN,NaN,NaN,0.000051,NaN
5,2020-12-20 08:35:00,0.003922,0.003923,0.003922,0.003922,105150.0,NaN,NaN,NaN,NaN,NaN,-0.000051,NaN
6,2020-12-20 08:36:00,0.003917,0.003917,0.003915,0.003915,81806.0,NaN,NaN,NaN,NaN,NaN,-0.000587,NaN
7,2020-12-20 08:37:00,0.003922,0.003926,0.003922,0.003923,752872.0,NaN,NaN,NaN,NaN,NaN,0.000382,NaN
8,2020-12-20 08:38:00,0.003922,0.003922,0.003920,0.003920,220340.0,NaN,NaN,NaN,NaN,NaN,-0.000484,NaN
9,2020-12-20 08:39:00,0.003920,0.003920,0.003920,0.003920,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
